In [1]:
import pandas as pd
import numpy as np
import json

from collections import defaultdict
from pandas.api.types import is_numeric_dtype

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk
from spacy.lang.en import English

from nltk import word_tokenize
import re
from keras.preprocessing.text import Tokenizer
from nltk.stem.porter import PorterStemmer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
model = SentimentIntensityAnalyzer()

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from keras.preprocessing.sequence import pad_sequences



Using TensorFlow backend.


# Reviews Dataset

In [2]:
children = pd.read_json('/Users/kritya/Downloads/Data/goodreads/reviews/goodreads_reviews_children.json', lines=True, nrows = 50000)
comics = pd.read_json('/Users/kritya/Downloads/Data/goodreads/reviews/goodreads_reviews_comics_graphic.json', lines=True, nrows = 50000)
fantasy = pd.read_json('/Users/kritya/Downloads/Data/goodreads/reviews/goodreads_reviews_fantasy_paranormal.json', lines=True, nrows = 50000)
history = pd.read_json('/Users/kritya/Downloads/Data/goodreads/reviews/goodreads_reviews_history_biography.json', lines=True, nrows = 50000)
poetry = pd.read_json('/Users/kritya/Downloads/Data/goodreads/reviews/goodreads_reviews_poetry.json', lines=True, nrows = 50000)
romance = pd.read_json('/Users/kritya/Downloads/Data/goodreads/reviews/goodreads_reviews_romance-2.json', lines=True, nrows = 50000)
young = pd.read_json('/Users/kritya/Downloads/Data/goodreads/reviews/goodreads_reviews_young_adult-2.json', lines=True, nrows = 50000)
mystery = pd.read_json('/Users/kritya/Downloads/Data/goodreads/reviews/goodreads_reviews_mystery_thriller_crime.json', lines=True, nrows = 50000)

# Book Details Dataset

In [3]:
children_books = pd.read_json('/Users/kritya/Downloads/Data/goodreads/books/goodreads_books_children.json', lines=True, nrows = 500)
comics_books = pd.read_json('/Users/kritya/Downloads/Data/goodreads/books/goodreads_books_comics_graphic.json', lines=True, nrows = 500)
fantasy_books = pd.read_json('/Users/kritya/Downloads/Data/goodreads/books/goodreads_books_fantasy_paranormal.json', lines=True, nrows = 500)
history_books = pd.read_json('/Users/kritya/Downloads/Data/goodreads/books/goodreads_books_history_biography.json', lines=True, nrows = 500)
poetry_books = pd.read_json('/Users/kritya/Downloads/Data/goodreads/books/goodreads_books_poetry.json', lines=True, nrows = 500)
romance_books = pd.read_json('/Users/kritya/Downloads/Data/goodreads/books/goodreads_books_romance.json', lines=True, nrows = 500)
young_books = pd.read_json('/Users/kritya/Downloads/Data/goodreads/books/goodreads_books_young_adult.json', lines=True, nrows = 500)
mystery_books = pd.read_json('/Users/kritya/Downloads/Data/goodreads/books/goodreads_books_mystery_thriller_crime.json', lines=True, nrows = 500)

In [4]:
children.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,23310161,f4b4b050f4be00e9283c92a814af2670,4,Fun sequel to the original.,Tue Nov 17 11:37:35 -0800 2015,Tue Nov 17 11:38:05 -0800 2015,,,7,0
1,8842281e1d1347389f2ab93d60773d4d,17290220,22d424a2b0057b18fb6ecf017af7be92,5,One of my favorite books to read to my 5 year ...,Sat Nov 08 08:54:03 -0800 2014,Wed Jan 25 13:56:12 -0800 2017,Tue Jan 24 00:00:00 -0800 2017,,4,0
2,8842281e1d1347389f2ab93d60773d4d,6954929,50ed4431c451d5677d98dd25ca8ec106,5,One of the best and most imaginative childrens...,Thu Oct 23 13:46:20 -0700 2014,Thu Oct 23 13:47:00 -0700 2014,,,6,1
3,8842281e1d1347389f2ab93d60773d4d,460548,1e4de11dd4fa4b7ffa59b6c69a6b28e9,5,My daughter is loving this. Published in the 6...,Mon Dec 02 10:43:59 -0800 2013,Wed Mar 22 11:47:25 -0700 2017,,,5,4
4,8842281e1d1347389f2ab93d60773d4d,11474551,2065145714bf747083a1c9ce81d5c4fe,5,A friend sent me this. Hilarious!,Wed May 11 22:38:11 -0700 2011,Sun Jan 29 15:56:41 -0800 2012,Wed May 11 00:00:00 -0700 2011,Wed May 11 00:00:00 -0700 2011,5,0


In [5]:
children.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       50000 non-null  object
 1   book_id       50000 non-null  int64 
 2   review_id     50000 non-null  object
 3   rating        50000 non-null  int64 
 4   review_text   50000 non-null  object
 5   date_added    50000 non-null  object
 6   date_updated  50000 non-null  object
 7   read_at       50000 non-null  object
 8   started_at    50000 non-null  object
 9   n_votes       50000 non-null  int64 
 10  n_comments    50000 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 4.2+ MB


In [6]:
children_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   isbn                  500 non-null    object 
 1   text_reviews_count    500 non-null    int64  
 2   series                500 non-null    object 
 3   country_code          500 non-null    object 
 4   language_code         500 non-null    object 
 5   popular_shelves       500 non-null    object 
 6   asin                  500 non-null    object 
 7   is_ebook              500 non-null    object 
 8   average_rating        500 non-null    float64
 9   kindle_asin           500 non-null    object 
 10  similar_books         500 non-null    object 
 11  description           500 non-null    object 
 12  format                500 non-null    object 
 13  link                  500 non-null    object 
 14  authors               500 non-null    object 
 15  publisher             5

In [7]:
children_books

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,1599150603,7,[],US,,"[{'count': '56', 'name': 'to-read'}, {'count':...",,false,4.13,B00DU10PUG,...,9,,2006,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,278578,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls
1,1934876569,6,[151854],US,,"[{'count': '515', 'name': 'to-read'}, {'count'...",,false,4.22,,...,3,,2009,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98,701117,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,...,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven
3,0915190575,4,[],US,,"[{'count': '8', 'name': 'to-read'}, {'count': ...",,false,4.29,,...,,,,https://www.goodreads.com/book/show/3209312-mo...,https://s.gr-assets.com/assets/nophoto/book/11...,3209312,11,3242879,"Moths and Mothers, Feathers and Fathers: A Sto...","Moths and Mothers, Feathers and Fathers: A Sto..."
4,1416904999,4,[],US,,"[{'count': '8', 'name': 'to-read'}, {'count': ...",,false,3.57,,...,6,,2005,https://www.goodreads.com/book/show/1698376.Wh...,https://s.gr-assets.com/assets/nophoto/book/11...,1698376,23,1695373,What Do You Do?,What Do You Do?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0590554425,8,"[146063, 1014021]",US,,"[{'count': '495', 'name': 'to-read'}, {'count'...",,false,3.73,,...,9,,1993,https://www.goodreads.com/book/show/125691.Say...,https://images.gr-assets.com/books/1358600904m...,125691,76,121027,"Say Cheese and Die! (Goosebumps, #4)","Say Cheese and Die! (Goosebumps, #4)"
496,0989162419,9,[],US,,"[{'count': '3', 'name': 'children-s-books'}, {...",,false,3.78,,...,10,,2015,https://www.goodreads.com/book/show/27071018-w...,https://images.gr-assets.com/books/1445100735m...,27071018,9,47113419,Why Horses Are,Why Horses Are
497,054573312X,21,[],US,,"[{'count': '72', 'name': 'to-read'}, {'count':...",,false,4.21,B00M9M40FI,...,2,,2015,https://www.goodreads.com/book/show/22749512-w...,https://images.gr-assets.com/books/1409185284m...,22749512,109,42294892,What If You Had Animal Feet?,What If You Had Animal Feet?
498,,3,[],US,eng,"[{'count': '6', 'name': 'to-read'}, {'count': ...",,false,4.16,,...,,,1959,https://www.goodreads.com/book/show/10000959-y...,https://images.gr-assets.com/books/1293145532m...,10000959,12,1611463,You Will Go to the Moon,You Will Go to the Moon


In [8]:
children_reviews = children.drop(['date_updated', 'read_at', 'n_votes', 'n_comments', 'started_at', 'rating', 'date_added'], axis = 1)
comics_reviews = comics.drop(['date_updated', 'read_at', 'n_votes', 'n_comments', 'started_at', 'rating', 'date_added'], axis = 1)
fantasy_reviews = fantasy.drop(['date_updated', 'read_at', 'n_votes', 'n_comments', 'started_at', 'rating', 'date_added'], axis = 1)
history_reviews = history.drop(['date_updated', 'read_at', 'n_votes', 'n_comments', 'started_at', 'rating', 'date_added'], axis = 1)
poetry_reviews = poetry.drop(['date_updated', 'read_at', 'n_votes', 'n_comments', 'started_at', 'rating', 'date_added'], axis = 1)
romance_reviews = romance.drop(['date_updated', 'read_at', 'n_votes', 'n_comments', 'started_at', 'rating', 'date_added'], axis = 1)
young_reviews = young.drop(['date_updated', 'read_at', 'n_votes', 'n_comments', 'started_at', 'rating', 'date_added'], axis = 1)
mystery_reviews = mystery.drop(['date_updated', 'read_at', 'n_votes', 'n_comments', 'started_at', 'rating', 'date_added'], axis = 1)

In [9]:
final_children_books = children_books.drop(['country_code', 'popular_shelves', 'asin', 'is_ebook', 'kindle_asin', 'format', 'publication_month', 'edition_information', 'publication_year', 'title_without_series', 'isbn', 'series', 'isbn13', 'similar_books', 'authors', 'work_id', 'text_reviews_count', 'publisher', 'num_pages', 'publication_day', 'ratings_count'], axis = 1)
final_comics_books = comics_books.drop(['country_code', 'popular_shelves', 'asin', 'is_ebook', 'kindle_asin', 'format', 'publication_month', 'edition_information', 'publication_year', 'title_without_series', 'isbn', 'series', 'isbn13', 'similar_books', 'authors', 'work_id', 'text_reviews_count', 'publisher', 'num_pages', 'publication_day', 'ratings_count'], axis = 1)
final_fantasy_books = fantasy_books.drop(['country_code', 'popular_shelves', 'asin', 'is_ebook', 'kindle_asin', 'format', 'publication_month', 'edition_information', 'publication_year', 'title_without_series', 'isbn', 'series', 'isbn13', 'similar_books', 'authors', 'work_id', 'text_reviews_count', 'publisher', 'num_pages', 'publication_day', 'ratings_count'], axis = 1)
final_history_books = history_books.drop(['country_code', 'popular_shelves', 'asin', 'is_ebook', 'kindle_asin', 'format', 'publication_month', 'edition_information', 'publication_year', 'title_without_series', 'isbn', 'series', 'isbn13', 'similar_books', 'authors', 'work_id', 'text_reviews_count', 'publisher', 'num_pages', 'publication_day', 'ratings_count'], axis = 1)
final_poetry_books = poetry_books.drop(['country_code', 'popular_shelves', 'asin', 'is_ebook', 'kindle_asin', 'format', 'publication_month', 'edition_information', 'publication_year', 'title_without_series', 'isbn', 'series', 'isbn13', 'similar_books', 'authors', 'work_id', 'text_reviews_count', 'publisher', 'num_pages', 'publication_day', 'ratings_count'], axis = 1)
final_romance_books = romance_books.drop(['country_code', 'popular_shelves', 'asin', 'is_ebook', 'kindle_asin', 'format', 'publication_month', 'edition_information', 'publication_year', 'title_without_series', 'isbn', 'series', 'isbn13', 'similar_books', 'authors', 'work_id', 'text_reviews_count', 'publisher', 'num_pages', 'publication_day', 'ratings_count'], axis = 1)
final_young_books = young_books.drop(['country_code', 'popular_shelves', 'asin', 'is_ebook', 'kindle_asin', 'format', 'publication_month', 'edition_information', 'publication_year', 'title_without_series', 'isbn', 'series', 'isbn13', 'similar_books', 'authors', 'work_id', 'text_reviews_count', 'publisher', 'num_pages', 'publication_day', 'ratings_count'], axis = 1)
final_mystery_books = mystery_books.drop(['country_code', 'popular_shelves', 'asin', 'is_ebook', 'kindle_asin', 'format', 'publication_month', 'edition_information', 'publication_year', 'title_without_series', 'isbn', 'series', 'isbn13', 'similar_books', 'authors', 'work_id', 'text_reviews_count', 'publisher', 'num_pages', 'publication_day', 'ratings_count'], axis = 1)

In [10]:
# final_children_books, final_comics_books, final_fantasy_books, final_history_books, final_poetry_books, final_romance_books, final_young_books, final_mystery_books


In [11]:
# children, comics, fantasy, history, poetry, romance, young, mystery 

In [12]:
children_reviews

,user_id,book_id,review_id,review_text
0,8842281e1d1347389f2ab93d60773d4d,23310161,f4b4b050f4be00e9283c92a814af2670,Fun sequel to the original.
1,8842281e1d1347389f2ab93d60773d4d,17290220,22d424a2b0057b18fb6ecf017af7be92,One of my favorite books to read to my 5 year ...
2,8842281e1d1347389f2ab93d60773d4d,6954929,50ed4431c451d5677d98dd25ca8ec106,One of the best and most imaginative childrens...
3,8842281e1d1347389f2ab93d60773d4d,460548,1e4de11dd4fa4b7ffa59b6c69a6b28e9,My daughter is loving this. Published in the 6...
4,8842281e1d1347389f2ab93d60773d4d,11474551,2065145714bf747083a1c9ce81d5c4fe,A friend sent me this. Hilarious!
...,...,...,...,...
49995,9003d274774f4c47e62f77600b08ac1d,17290240,65e2b773cc49eb2f720475d993536255,While it's true that some things are simply me...
49996,9003d274774f4c47e62f77600b08ac1d,17165923,a5f713d41ef307d4ceee474b2317252c,As best friends Mouse and Mole prepare Valenti...
49997,9003d274774f4c47e62f77600b08ac1d,15861457,4ba03821ae6d75134b35f40bbeef0551,The sequel to the charming Little Bear's Littl...
49998,9003d274774f4c47e62f77600b08ac1d,16719809,29af85acc8d644089c9de4cdf9b285cd,A wily and hungry fox slips into a nearby henh...


In [13]:
final_children_books

,language_code,average_rating,description,link,url,image_url,book_id,title
0,,4.13,"Relates in vigorous prose the tale of Aeneas, ...",https://www.goodreads.com/book/show/287141.The...,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,The Aeneid for Boys and Girls
1,,4.22,"To Kara's astonishment, she discovers that a p...",https://www.goodreads.com/book/show/6066812-al...,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,All's Fairy in Love and War (Avalon: Web of Ma...
2,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven
3,,4.29,,https://www.goodreads.com/book/show/3209312-mo...,https://www.goodreads.com/book/show/3209312-mo...,https://s.gr-assets.com/assets/nophoto/book/11...,3209312,"Moths and Mothers, Feathers and Fathers: A Sto..."
4,,3.57,WHAT DO YOU DO?\nA hen lays eggs...\nA cow giv...,https://www.goodreads.com/book/show/1698376.Wh...,https://www.goodreads.com/book/show/1698376.Wh...,https://s.gr-assets.com/assets/nophoto/book/11...,1698376,What Do You Do?
...,...,...,...,...,...,...,...,...
495,,3.73,"One of a series featuring Greg, who thinks it'...",https://www.goodreads.com/book/show/125691.Say...,https://www.goodreads.com/book/show/125691.Say...,https://images.gr-assets.com/books/1358600904m...,125691,"Say Cheese and Die! (Goosebumps, #4)"
496,,3.78,"It s not easy to give up something you love, b...",https://www.goodreads.com/book/show/27071018-w...,https://www.goodreads.com/book/show/27071018-w...,https://images.gr-assets.com/books/1445100735m...,27071018,Why Horses Are
497,,4.21,"If you could have any animal's feet, whose wou...",https://www.goodreads.com/book/show/22749512-w...,https://www.goodreads.com/book/show/22749512-w...,https://images.gr-assets.com/books/1409185284m...,22749512,What If You Had Animal Feet?
498,eng,4.16,"Originally published in 1959, prior to the fir...",https://www.goodreads.com/book/show/10000959-y...,https://www.goodreads.com/book/show/10000959-y...,https://images.gr-assets.com/books/1293145532m...,10000959,You Will Go to the Moon


# Selecting english books and merging documents

In [14]:
# Reviews = [children, comics, fantasy, history, poetry, romance, young, mystery]
# Books = [children_books, comics_books, fantasy_books, history_books, poetry_books, romance_books, young_books, mystery_books]
# Books

In [15]:
options = ['eng'] 
    
Children_books = final_children_books[final_children_books['language_code'].isin(options)].head(1000)
print("Done")
Comics_books = final_comics_books[final_comics_books['language_code'].isin(options)].head(1000)
print("Done")
Fantasy_books = final_fantasy_books[final_fantasy_books['language_code'].isin(options)].head(1000)
print("Done")
History_books = final_history_books[final_history_books['language_code'].isin(options)].head(1000)
print("Done")
Poetry_books = final_poetry_books[final_poetry_books['language_code'].isin(options)].head(1000)
print("Done")
Romance_books = final_romance_books[final_romance_books['language_code'].isin(options)].head(1000)
print("Done")
Young_books = final_young_books[final_young_books['language_code'].isin(options)].head(1000)
print("Done")
Mystery_books = final_mystery_books[final_mystery_books['language_code'].isin(options)].head(1000)
print("Done")
    

Done
Done
Done
Done
Done
Done
Done
Done


In [16]:
Mystery_books

,language_code,average_rating,description,link,url,image_url,book_id,title
1,eng,4.33,,https://www.goodreads.com/book/show/33394837-t...,https://www.goodreads.com/book/show/33394837-t...,https://images.gr-assets.com/books/1493114742m...,33394837,The House of Memory (Pluto's Snitch #2)
2,eng,3.49,"BATHS, BANKS AND ROMAN INSURRECTION\nDetective...",https://www.goodreads.com/book/show/29074697-t...,https://www.goodreads.com/book/show/29074697-t...,https://s.gr-assets.com/assets/nophoto/book/11...,29074697,The Slaughtered Virgin of Zenopolis (Inspector...
5,eng,3.96,Agatha Christie's most ingenious murder myster...,https://www.goodreads.com/book/show/16158998-a...,https://www.goodreads.com/book/show/16158998-a...,https://images.gr-assets.com/books/1360572193m...,16158998,A Murder is Announced
8,eng,4.14,"""Poetic, down trodden and nihilistic, Jonathan...",https://www.goodreads.com/book/show/22722787-t...,https://www.goodreads.com/book/show/22722787-t...,https://s.gr-assets.com/assets/nophoto/book/11...,22722787,The Cost of Doing Business
9,eng,3.93,In Agatha Christie's classic mystery 4:50 From...,https://www.goodreads.com/book/show/16158996-4,https://www.goodreads.com/book/show/16158996-4,https://images.gr-assets.com/books/1360566349m...,16158996,4:50 From Paddington
...,...,...,...,...,...,...,...,...
486,eng,4.16,"""Travis McGee is back in action and he is in f...",https://www.goodreads.com/book/show/761817.The...,https://www.goodreads.com/book/show/761817.The...,https://images.gr-assets.com/books/1324202163m...,761817,The Lonely Silver Rain (Travis McGee #21)
488,eng,4.45,*This is book 32 in the Frosted Love Cozy Myst...,https://www.goodreads.com/book/show/29081510-r...,https://www.goodreads.com/book/show/29081510-r...,https://s.gr-assets.com/assets/nophoto/book/11...,29081510,Red Hot Velvet Murder (Frosted Love Cozy Myste...
494,eng,3.73,"Hot water is nothing new for Stan Kraychik, a ...",https://www.goodreads.com/book/show/1609075.A_...,https://www.goodreads.com/book/show/1609075.A_...,https://s.gr-assets.com/assets/nophoto/book/11...,1609075,A Body to Dye For (Stan Kraychik Mystery #1)
496,eng,3.82,The world of Patricia Highsmith has always bee...,https://www.goodreads.com/book/show/15677.Stra...,https://www.goodreads.com/book/show/15677.Stra...,https://images.gr-assets.com/books/1331234879m...,15677,Strangers on a Train


In [17]:
Chi = Children_books[['book_id', 'language_code']]
Com = Comics_books[['book_id', 'language_code']]
Fan = Fantasy_books[['book_id', 'language_code']]
His = History_books[['book_id', 'language_code']]
Poe = Poetry_books[['book_id', 'language_code']]
Rom = Romance_books[['book_id', 'language_code']]
You = Young_books[['book_id', 'language_code']]
Mys = Mystery_books[['book_id', 'language_code']]

In [18]:
Ch = children_reviews.merge(Chi, how='inner', on='book_id')
Co = comics_reviews.merge(Com, how='inner', on='book_id')
Fa = fantasy_reviews.merge(Fan, how='inner', on='book_id')
Hi = history_reviews.merge(His, how='inner', on='book_id')
Po =  poetry_reviews.merge(Poe, how='inner', on='book_id')
Ro =  romance_reviews.merge(Rom, how='inner', on='book_id')
Yo =  young_reviews.merge(You, how='inner', on='book_id')
My =  mystery_reviews.merge(Mys, how='inner', on='book_id')

In [19]:
Po

,user_id,book_id,review_id,review_text,language_code
0,7b0457e81e3cc2b6180238760d608728,30353202,2ee0660908e0596394e729043de5b5d7,When the Moon Takes Over the Dream is a collec...,eng
1,3731e18793333d8e443a557388cb9579,30353202,c65fbbe8cc482daf3ce08be4df5996e9,To see this review and to learn more about thi...,eng
2,b4bd433ebcab24e0b8595f71858b7be1,30353202,31163592d98a409bc096e5ba6609d817,"Another series of deep, thought provoking, bea...",eng
3,1bff2fc0d8d1c8a5baf2a676b681e3d6,30353202,951d19a8b4b218031c4b05389cf4919f,"This is a book full of beautiful, romantic poe...",eng
4,71e14e3befa01f31723572d437dbda76,30353202,63413dc0632fa1384f2be271acfa5ce6,When the Moon takes over the Dream: Love Poetr...,eng
...,...,...,...,...,...
166,d9b9a031915e4caf6eb4a9174d8ed112,6580847,b67935c24368797b4f5d2160c1361e97,"Wallace Stevens is one of the best poets, not ...",eng
167,10930094a2abe628bf3be23f3f95277c,6324818,983482f91cf389b9c26b2702d1fa35f1,"At first, I was impressed with Agard's attempt...",eng
168,934a6e67b3040d63b24533ee7a0ecae7,24940289,6e812ebb102836ae52c3ca290833df89,The Library of America published this volume o...,eng
169,a971633a58c8690f37720fccb0be8d46,18406193,4a1656e187ad7aa284d0d87b6bd0f600,THIS REVIEW FIRST APPEARED IN THE COMPULSIVE R...,eng


In [21]:
# Children.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/Children.csv', index = False, header=True)
# Comics.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/Comics.csv', index = False, header=True)
# Fantasy.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/Fantasy.csv', index = False, header=True)
# History.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/History.csv', index = False, header=True)
# Poetry.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/Poetry.csv', index = False, header=True)
# Romance.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/Romance.csv', index = False, header=True)
# Young.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/Young.csv', index = False, header=True)
# Mystery.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/Mystery.csv', index = False, header=True)

In [27]:
# Children = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Children.csv')
# Comics = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Comics.csv')
# Fantasy = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Fantasy.csv')
# History = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/History.csv')
# Poetry = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Poetry.csv')
# Romance = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Romance.csv')
# Young = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Young.csv')
# Mystery = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Mystery.csv')

In [28]:
Ch['Genre'] = 'Children'
Co['Genre'] = 'Comics'
Fa['Genre'] = 'Fantasy'
Hi['Genre'] = 'History'
Po['Genre'] = 'Poetry'
Ro['Genre'] = 'Romance'
Yo['Genre'] = 'Young'
My['Genre'] = 'Mystery'

In [65]:
Children = final_children_books.merge(Ch, how='inner', on='book_id')
Comics = final_comics_books.merge(Co, how='inner', on='book_id')
Fantasy = final_fantasy_books.merge(Fa, how='inner', on='book_id')
History = final_history_books.merge(Hi, how='inner', on='book_id')
Poetry =  final_poetry_books.merge(Po, how='inner', on='book_id')
Romance =  final_romance_books.merge(Ro, how='inner', on='book_id')
Young =  final_young_books.merge(Yo, how='inner', on='book_id')
Mystery =  final_mystery_books.merge(My, how='inner', on='book_id')

In [66]:
Children

,language_code_x,average_rating,description,link,url,image_url,book_id,title,user_id,review_id,review_text,language_code_y,Genre,processed_text,sentiment_score
0,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven,3e83e94eeea96cb79841f1bb8292ab8a,93824fba45f41359562e8963df098115,"b'Really cute, sweet, and charming.'",eng,Children,really cute sweet and charming,0.8852
1,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven,c10765c997317e4cf75aecdad046fd72,bfdb7f3d1ca07e8fa4a3ba6a412a9048,"b""This beautiful children's book was anonymous...",eng,Children,this beautiful child s book wa anonymously giv...,0.9899
2,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven,cfe4738a5d926fc158d4fcb1287fe2c5,96d235a1118ca744a2da86c139253de3,b'A really nice book for anyone that has lost ...,eng,Children,a really nice book for anyone that ha lost a b...,0.7574
3,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven,6ddac89eaa68f406f1a5f2d39ab384f1,2d71688484b7e3fc52b96f4413b3e689,b'Another book which I picked out solely on th...,eng,Children,another book which i picked out solely on the ...,0.6865
4,eng,3.75,,https://www.goodreads.com/book/show/2519710.Tulku,https://www.goodreads.com/book/show/2519710.Tulku,https://images.gr-assets.com/books/1327783033m...,2519710,Tulku,b382989c8c4eaa41b0cd1298e0d401e7,cf821c8e248725841ea1a31b96347213,"b""Thirteen year old Theo is hiding in the wood...",eng,Children,thirteen year old theo is hiding in the wood d...,0.5994
5,eng,3.84,Featuring the adventures of five children - Ed...,https://www.goodreads.com/book/show/11881251-d...,https://www.goodreads.com/book/show/11881251-d...,https://images.gr-assets.com/books/1309370057m...,11881251,Dream Days,689fdf3048dc379ab1e617f950e3f8b0,35ae8e44d4ec5204241b045e5e4adb7c,b'Fra den gyldne tidsalder for bornelitteratur...,eng,Children,fra den gyldne tidsalder for bornelitteratur k...,0.5719
6,eng,4.44,The much-anticipated follow-up to the E. B. Wh...,https://www.goodreads.com/book/show/13588160-i...,https://www.goodreads.com/book/show/13588160-i...,https://images.gr-assets.com/books/1351267631m...,13588160,If I Built a House,51f2623b0a4a7ef500aa08d2a2224e1e,2cbf1f1324504e510905afdec00a9f1f,"b""Much of what I wrote for https://www.goodrea...",eng,Children,much of what i wrote for http www goodreads co...,0.9468
7,eng,4.44,The much-anticipated follow-up to the E. B. Wh...,https://www.goodreads.com/book/show/13588160-i...,https://www.goodreads.com/book/show/13588160-i...,https://images.gr-assets.com/books/1351267631m...,13588160,If I Built a House,7af826448b8bff78ec284a3e77f8e187,871cb130608f5e956b8aba7d4de4c950,b'This house is probably similar to those most...,eng,Children,this house is probably similar to those most o...,0.9246
8,eng,4.44,The much-anticipated follow-up to the E. B. Wh...,https://www.goodreads.com/book/show/13588160-i...,https://www.goodreads.com/book/show/13588160-i...,https://images.gr-assets.com/books/1351267631m...,13588160,If I Built a House,b9e95e45d4160573e8bc8695b0cf3b54,a3908624b506fe378bd4ca7669e6f67d,"b""I love, love, love this book. It's stunning ...",eng,Children,i love love love this book it s stunning illus...,0.9862
9,eng,4.17,"Dink, Josh, And Ruth Rose are back! This time,...",https://www.goodreads.com/book/show/2161796.De...,https://www.goodreads.com/book/show

In [80]:
children_book_titles = pd.unique(Children['title'])
children_book_titles

array(['Dog Heaven', 'Tulku', 'Dream Days', 'If I Built a House',
       'Detective Camp (A to Z Mysteries: Super Edition, #1)',
       'The Kidnapped King (A to Z Mysteries, #11)',
       "Anno's Counting Book",
       'The Librarian from the Black Lagoon (Black Lagoon, #5)',
       'Ghost Knight', 'Cinderella, or The Little Glass Slipper',
       'Ada Byron Lovelace and the Thinking Machine',
       'The Secret Chicken Society',
       'The Happy Hollisters at Circus Island (Happy Hollisters, #8)',
       "Hank's Big Day: The Story of a Bug",
       'The True Blue Scouts of Sugar Man Swamp',
       'The Miraculous Journey of Edward Tulane',
       'Wayside School Gets A Little Stranger (Wayside School #3)',
       'Scaredy Kat (Suddenly Supernatural #2)',
       'Dawn and the Older Boy (The Baby-Sitters Club, #37)',
       'Canterbury Tales',
       'The Candymakers and the Great Chocolate Chase',
       'Peter Pan & Peter Pan in Kensington Gardens', 'Heidi',
       'Mud-Puddle Poodl

In [83]:
# children, comics, fantasy, history, poetry, romance, young, mystery
Children_book_stats = Children.groupby('title').agg({'average_rating': [np.size, np.mean]})
Comics_book_stats = Comics.groupby('title').agg({'average_rating': [np.size, np.mean]})
Fantasy_book_stats = Fantasy.groupby('title').agg({'average_rating': [np.size, np.mean]})
History_book_stats = History.groupby('title').agg({'average_rating': [np.size, np.mean]})
Poetry_book_stats = Poetry.groupby('title').agg({'average_rating': [np.size, np.mean]})
Romance_book_stats = Romance.groupby('title').agg({'average_rating': [np.size, np.mean]})
Young_book_stats = Young.groupby('title').agg({'average_rating': [np.size, np.mean]})
Mystery_book_stats = Mystery.groupby('title').agg({'average_rating': [np.size, np.mean]})

In [87]:
min_20 = Mystery_book_stats['average_rating']['size'] >= 5
Mystery_book_stats[min_20].sort_values([('average_rating', 'mean')], ascending=False)

average_rating      
                                                size  mean
title                                                     
Every Dead Thing (Charlie Parker, #1)              5  4.01
Strangers on a Train                               7  3.82
Flesh and Blood (Kay Scarpetta, #22)               5  3.67
There's Someone Inside Your House                  9  3.44
Eyes on You                                        9  3.13

# Initial Recommendation List (Item-Item)

# Sentiment Analysis

In [99]:
# Pre-processing

from nltk import pos_tag, word_tokenize
def posTag(review):
    tagged_text_list=[]
    for text in review:
        tagged_text_list.append(pos_tag(word_tokenize(text)))
    return tagged_text_list

def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
        for word,tag in text_list:
            if tag in ['NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
                final_text.append(word)
        final_text_list.append(' '.join(final_text))
    return final_text_list

def clean_reviews(review):  
    
    # 1. Removing html tags
    text = BeautifulSoup(review,"lxml").get_text()
    
    # 2. Retaining only alphabets.
    Text = re.sub("[^a-zA-Z]"," ",text)
    
    # 3. Converting to lower case and splitting
    word_tokens= Text.lower().split()

    # 4. POS tagging and filtering
    #word_tokens = posTag(word_tokens)
    #word_tokens = filterTag(word_tokens)
    
    # 5. Lemmetization
    le=WordNetLemmatizer()
    stop_words= set(stopwords.words("english"))     
    word_tokens= [le.lemmatize(w) for w in word_tokens]
    
    cleaned_review=" ".join(word_tokens)
    return cleaned_review

In [100]:
Ch['review_text'] = Ch['review_text'].astype('|S')
Co['review_text'] = Co['review_text'].astype('|S')
Fa['review_text'] = Fa['review_text'].astype('|S')
Hi['review_text'] = Hi['review_text'].astype('|S')
Po['review_text'] = Po['review_text'].astype('|S')
Ro['review_text'] = Ro['review_text'].astype('|S')
Yo['review_text'] = Yo['review_text'].astype('|S')
My['review_text'] = My['review_text'].astype('|S')

In [101]:
#All_Books['processed_text']= All_Books['review_text'].apply(clean_reviews)

Ch['processed_text'] = Ch['review_text'].apply(clean_reviews)
Co['processed_text'] = Co['review_text'].apply(clean_reviews)
Fa['processed_text'] = Fa['review_text'].apply(clean_reviews)
Hi['processed_text'] = Hi['review_text'].apply(clean_reviews)
Po['processed_text'] = Po['review_text'].apply(clean_reviews)
Ro['processed_text'] = Ro['review_text'].apply(clean_reviews)
Yo['processed_text'] = Yo['review_text'].apply(clean_reviews)
My['processed_text'] = My['review_text'].apply(clean_reviews)

/Users/kritya/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://www.bookcrossing.com/journal/1..." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/kritya/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://www.pussreboots.pair.com/blog/..." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/kritya/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://www.segredosemlivros.com/2016/..." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the

In [111]:
Children

,language_code_x,average_rating,description,link,url,image_url,book_id,title,user_id,review_id,review_text,language_code_y,Genre,processed_text,sentiment_score
0,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven,3e83e94eeea96cb79841f1bb8292ab8a,93824fba45f41359562e8963df098115,"b'Really cute, sweet, and charming.'",eng,Children,really cute sweet and charming,0.8852
1,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven,c10765c997317e4cf75aecdad046fd72,bfdb7f3d1ca07e8fa4a3ba6a412a9048,"b""This beautiful children's book was anonymous...",eng,Children,this beautiful child s book wa anonymously giv...,0.9899
2,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven,cfe4738a5d926fc158d4fcb1287fe2c5,96d235a1118ca744a2da86c139253de3,b'A really nice book for anyone that has lost ...,eng,Children,a really nice book for anyone that ha lost a b...,0.7574
3,eng,4.43,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,Dog Heaven,6ddac89eaa68f406f1a5f2d39ab384f1,2d71688484b7e3fc52b96f4413b3e689,b'Another book which I picked out solely on th...,eng,Children,another book which i picked out solely on the ...,0.6865
4,eng,3.75,,https://www.goodreads.com/book/show/2519710.Tulku,https://www.goodreads.com/book/show/2519710.Tulku,https://images.gr-assets.com/books/1327783033m...,2519710,Tulku,b382989c8c4eaa41b0cd1298e0d401e7,cf821c8e248725841ea1a31b96347213,"b""Thirteen year old Theo is hiding in the wood...",eng,Children,thirteen year old theo is hiding in the wood d...,0.5994
5,eng,3.84,Featuring the adventures of five children - Ed...,https://www.goodreads.com/book/show/11881251-d...,https://www.goodreads.com/book/show/11881251-d...,https://images.gr-assets.com/books/1309370057m...,11881251,Dream Days,689fdf3048dc379ab1e617f950e3f8b0,35ae8e44d4ec5204241b045e5e4adb7c,b'Fra den gyldne tidsalder for bornelitteratur...,eng,Children,fra den gyldne tidsalder for bornelitteratur k...,0.5719
6,eng,4.44,The much-anticipated follow-up to the E. B. Wh...,https://www.goodreads.com/book/show/13588160-i...,https://www.goodreads.com/book/show/13588160-i...,https://images.gr-assets.com/books/1351267631m...,13588160,If I Built a House,51f2623b0a4a7ef500aa08d2a2224e1e,2cbf1f1324504e510905afdec00a9f1f,"b""Much of what I wrote for https://www.goodrea...",eng,Children,much of what i wrote for http www goodreads co...,0.9468
7,eng,4.44,The much-anticipated follow-up to the E. B. Wh...,https://www.goodreads.com/book/show/13588160-i...,https://www.goodreads.com/book/show/13588160-i...,https://images.gr-assets.com/books/1351267631m...,13588160,If I Built a House,7af826448b8bff78ec284a3e77f8e187,871cb130608f5e956b8aba7d4de4c950,b'This house is probably similar to those most...,eng,Children,this house is probably similar to those most o...,0.9246
8,eng,4.44,The much-anticipated follow-up to the E. B. Wh...,https://www.goodreads.com/book/show/13588160-i...,https://www.goodreads.com/book/show/13588160-i...,https://images.gr-assets.com/books/1351267631m...,13588160,If I Built a House,b9e95e45d4160573e8bc8695b0cf3b54,a3908624b506fe378bd4ca7669e6f67d,"b""I love, love, love this book. It's stunning ...",eng,Children,i love love love this book it s stunning illus...,0.9862
9,eng,4.17,"Dink, Josh, And Ruth Rose are back! This time,...",https://www.goodreads.com/book/show/2161796.De...,https://www.goodreads.com/book/show

In [102]:
def sentiment_analysis(sentence):
    
    sentiment_dict = model.polarity_scores(sentence)
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing import sequence
    tokenizer = Tokenizer()
    indices = tokenizer.texts_to_sequences(sentence)
    indices = pad_sequences(indices, maxlen=609)
    
    overall = sentiment_dict['compound']
    return overall

In [103]:
#All_Books['review_text'] = All_Books['review_text'].astype('|S')
#All_Books['sentiment_score']= All_Books['processed_text'].apply(sentiment_analysis)

Ch['sentiment_score'] = Ch['processed_text'].apply(sentiment_analysis)
Co['sentiment_score'] = Co['processed_text'].apply(sentiment_analysis)
Fa['sentiment_score'] = Fa['processed_text'].apply(sentiment_analysis)
Hi['sentiment_score'] = Hi['processed_text'].apply(sentiment_analysis)
Po['sentiment_score'] = Po['processed_text'].apply(sentiment_analysis)
Ro['sentiment_score'] = Ro['processed_text'].apply(sentiment_analysis)
Yo['sentiment_score'] = Yo['processed_text'].apply(sentiment_analysis)
My['sentiment_score'] = My['processed_text'].apply(sentiment_analysis)
print('Done!!!')

Done!!!


In [104]:
Ch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          50 non-null     object 
 1   book_id          50 non-null     int64  
 2   review_id        50 non-null     object 
 3   review_text      50 non-null     |S1933 
 4   language_code    50 non-null     object 
 5   Genre            50 non-null     object 
 6   processed_text   50 non-null     object 
 7   sentiment_score  50 non-null     float64
dtypes: bytes15464(1), float64(1), int64(1), object(5)
memory usage: 97.5+ KB


In [105]:
children = Ch.drop(['book_id','review_id','processed_text', 'language_code'], axis = 1)
comics = Co.drop(['book_id','review_id','processed_text', 'language_code'], axis = 1)
fantasy = Fa.drop(['book_id','review_id','processed_text', 'language_code'], axis = 1)
history = Hi.drop(['book_id','review_id','processed_text', 'language_code'], axis = 1)
poetry = Po.drop(['book_id','review_id','processed_text', 'language_code'], axis = 1)
romance = Ro.drop(['book_id','review_id','processed_text', 'language_code'], axis = 1)
young = Yo.drop(['book_id','review_id','processed_text', 'language_code'], axis = 1)
mystery = My.drop(['book_id','review_id','processed_text', 'language_code'], axis = 1)

In [106]:
frames = [Ch, Co, Fa, Hi, Po, Ro, Yo, My]
 
All = pd.concat(frames, ignore_index=True)
All

,user_id,book_id,review_id,review_text,language_code,Genre,processed_text,sentiment_score
0,7b2e5fe9fd353fecf3eeebb4850b88d3,16074339,4f4371f0f6d47418da462e36f3406c4f,"b""Wow.......a very fast read, because you don'...",eng,Children,wow a very fast read because you don t want to...,0.7050
1,46e23d760843b8c9a4108a55cd92413d,16074339,20f45166118f19643938c164c936cc3d,"b""I have never liked animal stories, ever, and...",eng,Children,i have never liked animal story ever and i m n...,0.9937
2,d1e368a7d2870eb6fbf6e0d350568a2d,26722927,ccdb7d7b3672c9b0f49367b4c809397b,"b""Tina is a curious cow. She has 3 sisters who...",eng,Children,tina is a curious cow she ha sister who care a...,0.9667
3,832f0a426cc8990d9f40fba9567b442e,26722927,de8451c41af98b7e5c887173bc420e2d,b'This book is pretty darn adorable! I look fo...,eng,Children,this book is pretty darn adorable i look forwa...,0.8481
4,ab2fadb5c7bbe55c80406d2b3692e969,28118451,67183f570e468007bcb4330930fb3ebd,"b""4th&up \n Magical realism, a bit of a myster...",eng,Children,th up magical realism a bit of a mystery and a...,0.9904
...,...,...,...,...,...,...,...,...
657,14ec84f6ae39e82e95ee0d8c76ed30cb,29985598,d6eb233236d635fef7c04ce33bb095dd,"b'A cute, cozy mystery.'",eng,Mystery,a cute cozy mystery,0.4588
658,018fb46979eaef202859956d5ccb905b,77608,2fa4dd13e36164e08fab085dade52c22,b'Stout at his absolute best as Archie and Wol...,eng,Mystery,stout at his absolute best a archie and wolfe ...,0.9200
659,2159f55d397e8fbe68d5e03668e7d9d2,2161796,8e460d95ef144879fe441ab398352389,b'Read this in 2nd grade',eng,Mystery,read this in nd grade,0.0000
660,9e92361fe6fd80f6f6935bd1c8ccc8ac,28144874,ca7825f74e0dcb118ebc85cd64178034,"b""For someone who isn't a fan of Thriller/Horr...",eng,Mystery,for someone who isn t a fan of thriller horror...,0.9412


In [107]:
count = len(Ch)+len(Co)+len(Fa)+len(Hi)+len(Po)+len(Ro)+len(Yo)+len(My)
print(count)

662


In [108]:
Ch.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Children.csv', index = False, header=True)
Co.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Comics.csv', index = False, header=True)
Fa.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Fantasy.csv', index = False, header=True)
Hi.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/History.csv', index = False, header=True)
Po.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Poetry.csv', index = False, header=True)
Ro.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Romance.csv', index = False, header=True)
Yo.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Young.csv', index = False, header=True)
My.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Mystery.csv', index = False, header=True)
All.to_csv ('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Mystery.csv', index = False, header=True)

In [57]:
# Children = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Children.csv')
# Comics = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Comics.csv')
# Fantasy = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Fantasy.csv')
# History = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/History.csv')
# Poetry = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Poetry.csv')
# Romance = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Romance.csv')
# Young = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Young.csv')
# Mystery = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/PHP Data/Rev and Meta/Mystery.csv')

In [ ]:
# frames = [Children, Comics, Fantasy, History, Poetry, Romance, Young, Mystery]
 
# All_Genres = pd.concat(frames, ignore_index=True)
# All_Genres

In [ ]:
# All_Genres.to_csv ('/Users/kritya/Downloads/Data/goodreads/merged dataset/All Genres.csv', index = False, header=True)

In [ ]:
# Children

In [109]:
u = Children.title.unique()
print(len(u))

30


# Best Books List (Based on Sentiment Score on all books)

In [112]:
# List based on rating
Children_ratings = pd.DataFrame(Children.groupby('title')['sentiment_score'].mean())
Children_ratings['total score'] = pd.DataFrame(Children.groupby('title')['sentiment_score'].count())
Children_ratings = Children_ratings.sort_values('total score', ascending=False)

# List based on rating
Comics_ratings = pd.DataFrame(Comics.groupby('title')['sentiment_score'].mean())
Comics_ratings['total score'] = pd.DataFrame(Comics.groupby('title')['sentiment_score'].count())
Comics_ratings = Comics_ratings.sort_values('total score', ascending=False).head(6)


# List based on rating
History_ratings = pd.DataFrame(History.groupby('title')['sentiment_score'].mean())
History_ratings['total score'] = pd.DataFrame(History.groupby('title')['sentiment_score'].count())
History_ratings = History_ratings.sort_values('total score', ascending=False).head(6)


# List based on rating
Mystery_ratings = pd.DataFrame(Mystery.groupby('title')['sentiment_score'].mean())
Mystery_ratings['total score'] = pd.DataFrame(Mystery.groupby('title')['sentiment_score'].count())
Mystery_ratings = Mystery_ratings.sort_values('total score', ascending=False).head(6)


# List based on rating
Poetry_ratings = pd.DataFrame(Poetry.groupby('title')['sentiment_score'].mean())
Poetry_ratings['total score'] = pd.DataFrame(Poetry.groupby('title')['sentiment_score'].count())
Poetry_ratings = Poetry_ratings.sort_values('total score', ascending=False).head(6)


# List based on rating
Young_ratings = pd.DataFrame(Young.groupby('title')['sentiment_score'].mean())
Young_ratings['total score'] = pd.DataFrame(Young.groupby('title')['sentiment_score'].count())
Young_ratings = Young_ratings.sort_values('total score', ascending=False).head(6)

In [ ]:
Children_ratings.to_csv ('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Children_ratings.csv', index = True, header=True)
Comics_ratings.to_csv ('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Comics_ratings.csv', index = True, header=True)
History_ratings.to_csv ('/Users/kritya/Downloads/Data/goodreads/ranking dataset/History_ratings.csv', index = True, header=True)
Poetry_ratings.to_csv ('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Poetry_ratings.csv', index = True, header=True)
Young_ratings.to_csv ('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Young_ratings.csv', index = True, header=True)
Mystery_ratings.to_csv ('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Mystery_ratings.csv', index = True, header=True)

In [ ]:
Children = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Children.csv')
Comics = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Comics.csv')
Young = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Young.csv')
Mystery = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/merged dataset/Mystery.csv')

Children_ratings = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Children_ratings.csv')
Comics_ratings = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Comics_ratings.csv')
Young_ratings = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Young_ratings.csv')
Mystery_ratings = pd.read_csv('/Users/kritya/Downloads/Data/goodreads/ranking dataset/Mystery_ratings.csv')

In [ ]:
# List based on rating
All_ratings = pd.DataFrame(All_Genres.groupby('title')['sentiment_score'].mean())
All_ratings['total score'] = pd.DataFrame(All_Genres.groupby('title')['sentiment_score'].count())
value = All_ratings.sort_values('total score', ascending=False).head(6)

In [ ]:
# All_ratings.to_csv ('/Users/kritya/Downloads/Data/goodreads/ranking dataset/All_ratings.csv', index = True, header=True)

In [ ]:
Children_ratings

In [114]:
# List based on rating
df_n_ratings = pd.DataFrame(Comics.groupby('title')['sentiment_score'].mean())
df_n_ratings['total score'] = pd.DataFrame(Comics.groupby('title')['sentiment_score'].count())
df_n_ratings.rename(columns = {'Sentiment Score': 'mean score'}, inplace=True)

sorted_values = df_n_ratings.sort_values('total score', ascending=False).head(6)

In [115]:
sorted_values

,sentiment_score,total score
title,,
"Alex + Ada, Vol. 1",0.501010,39
Batman: Year One,0.422603,32
"Pretty Guardian Sailor Moon, Vol. 4 (Pretty Soldier Sailor Moon Renewal Edition, #4)",0.153675,4
"The Secret of the Unicorn (Tintin, #11)",0.764425,4
X-Force/Cable: Messiah War,0.563400,4
Habibi,0.844667,3


In [116]:
for i in range(0,6):
    BSB = Children[Children['title'] == sorted_values.index[i]].head(1)
    print(BSB['description'])

Series([], Name: description, dtype: object)
Series([], Name: description, dtype: object)
Series([], Name: description, dtype: object)
Series([], Name: description, dtype: object)
Series([], Name: description, dtype: object)
Series([], Name: description, dtype: object)


In [130]:
# List based on rating
df_n_ratings = pd.DataFrame(Children.groupby('title')['sentiment_score'].mean())
df_n_ratings['total ratings'] = pd.DataFrame(Children.groupby('title')['sentiment_score'].count())
df_n_ratings.rename(columns = {'rating': 'mean ratings'}, inplace=True)

df_n_ratings.sort_values('total ratings', ascending=False).head(50)

,sentiment_score,total ratings
title,,
Ghost Knight,0.274133,6
Dog Heaven,0.829750,4
Hank's Big Day: The Story of a Bug,0.923275,4
Peter Pan & Peter Pan in Kensington Gardens,0.638033,3
If I Built a House,0.952533,3
The True Blue Scouts of Sugar Man Swamp,0.849350,2
The Cow Who Climbed a Tree,0.907400,2
Wayside School Gets A Little Stranger (Wayside School #3),0.772500,2
"Cinderella, or The Little Glass Slipper",0.611300,2


In [131]:
userRatings = Children.pivot_table(index=['user_id'],columns=['title'],values='sentiment_score')
userRatings

title,Ada Byron Lovelace and the Thinking Machine,Anno's Counting Book,Canterbury Tales,"Cinderella, or The Little Glass Slipper","Clementine and the Spring Trip (Clementine, #6)","Dawn and the Older Boy (The Baby-Sitters Club, #37)","Detective Camp (A to Z Mysteries: Super Edition, #1)",Dog Heaven,Dream Days,Ghost Knight,...,"The Happy Hollisters at Circus Island (Happy Hollisters, #8)","The Kidnapped King (A to Z Mysteries, #11)","The Librarian from the Black Lagoon (Black Lagoon, #5)",The Miraculous Journey of Edward Tulane,The Secret Chicken Society,"The Templeton Twins Make a Scene (Templeton Twins, #2)",The True Blue Scouts of Sugar Man Swamp,The Very Hairy Bear.,Tulku,Wayside School Gets A Little Stranger (Wayside School #3)
user_id,,,,,,,,,,,,,,,,,,,,,
08d805375530cc208801531ca7fdefbc,NaN,NaN,NaN,NaN,NaN,-0.5719,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10b7d43ff18698e75103de17eab2b2cf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2159f55d397e8fbe68d5e03668e7d9d2,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246eac0ca4d02e73d8768a5acb9b89d7,NaN,NaN,0.9973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2cdc050a31e73ab040509f242727702f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.4215,NaN,NaN,NaN,NaN
2ce88e439b789bd2aa41745bee70552b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2d1ad7ff9163a860f5c9d2daa9e4b57e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385dfba36a344c6f631d9401a7132153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.8731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3e83e94eeea96cb79841f1bb8292ab8a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8852,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
item_util_matrix = userRatings.copy()
item_util_matrix = item_util_matrix.apply(lambda col : col.fillna(col.mean()), axis=0)
item_util_matrix.head(5)

## Avg rating is filled for all NaN

title,Ada Byron Lovelace and the Thinking Machine,Anno's Counting Book,Canterbury Tales,"Cinderella, or The Little Glass Slipper","Clementine and the Spring Trip (Clementine, #6)","Dawn and the Older Boy (The Baby-Sitters Club, #37)","Detective Camp (A to Z Mysteries: Super Edition, #1)",Dog Heaven,Dream Days,Ghost Knight,...,"The Happy Hollisters at Circus Island (Happy Hollisters, #8)","The Kidnapped King (A to Z Mysteries, #11)","The Librarian from the Black Lagoon (Black Lagoon, #5)",The Miraculous Journey of Edward Tulane,The Secret Chicken Society,"The Templeton Twins Make a Scene (Templeton Twins, #2)",The True Blue Scouts of Sugar Man Swamp,The Very Hairy Bear.,Tulku,Wayside School Gets A Little Stranger (Wayside School #3)
user_id,,,,,,,,,,,,,,,,,,,,,
08d805375530cc208801531ca7fdefbc,0.624,0.9278,0.9973,0.6113,0.74895,-0.5719,0.0,0.82975,0.5719,0.274133,...,0.8271,0.0,-0.4019,0.9118,0.9485,0.4215,0.84935,0.0049,0.5994,0.7725
10b7d43ff18698e75103de17eab2b2cf,0.624,0.9278,0.9973,0.6113,0.74895,-0.5719,0.0,0.82975,0.5719,0.274133,...,0.8271,0.0,-0.4019,0.9118,0.9485,0.4215,0.84935,0.0049,0.5994,0.7725
2159f55d397e8fbe68d5e03668e7d9d2,0.624,0.9278,0.9973,0.6113,0.74895,-0.5719,0.0,0.82975,0.5719,0.274133,...,0.8271,0.0,-0.4019,0.9118,0.9485,0.4215,0.84935,0.0049,0.5994,0.7725
246eac0ca4d02e73d8768a5acb9b89d7,0.624,0.9278,0.9973,0.6113,0.74895,-0.5719,0.0,0.82975,0.5719,0.274133,...,0.8271,0.0,-0.4019,0.9118,0.9485,0.4215,0.84935,0.0049,0.5994,0.7725
2cdc050a31e73ab040509f242727702f,0.624,0.9278,0.9973,0.6113,0.74895,-0.5719,0.0,0.82975,0.5719,0.274133,...,0.8271,0.0,-0.4019,0.9118,0.9485,0.4215,0.84935,0.0049,0.5994,0.7725


In [133]:
item_util_matrix.isna().sum().sum()

0

In [134]:
item_corr_matrix = item_util_matrix.corr()

In [135]:
book = 'Dog Heaven'

book_corr = item_corr_matrix[book]
book_corr = book_corr.sort_values(ascending=False)
book_corr.dropna(inplace=True)

In [137]:
similar_books = pd.DataFrame(data=book_corr.values, columns=['Correlation'], 
                                               index = book_corr.index)
similar_books = similar_books.join(df_n_ratings['total ratings'])
similar_books.head(6)

,Correlation,total ratings
title,,
Dog Heaven,1.000000e+00,4
"Clementine and the Spring Trip (Clementine, #6)",2.191865e-01,2
Hank's Big Day: The Story of a Bug,5.581820e-16,4
Peter Pan & Peter Pan in Kensington Gardens,4.211368e-16,3
The Cow Who Climbed a Tree,2.607281e-16,2
Ghost Knight,4.618706e-17,6


In [138]:
similar_books

,Correlation,total ratings
title,,
Dog Heaven,1.000000e+00,4
"Clementine and the Spring Trip (Clementine, #6)",2.191865e-01,2
Hank's Big Day: The Story of a Bug,5.581820e-16,4
Peter Pan & Peter Pan in Kensington Gardens,4.211368e-16,3
The Cow Who Climbed a Tree,2.607281e-16,2
Ghost Knight,4.618706e-17,6
The True Blue Scouts of Sugar Man Swamp,1.585178e-17,2
"Cinderella, or The Little Glass Slipper",-3.093239e-17,2
If I Built a House,-4.930325e-16,3
